In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
print("GPUs available:", tf.config.list_physical_devices('GPU'))

GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# MobileFaceNet Block Definitions
def conv_block(inputs, filters, kernel, strides, activation=True):
    x = layers.Conv2D(filters, kernel, strides=strides, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.PReLU(shared_axes=[1, 2])(x)
    return x

In [4]:
def depthwise_conv_block(inputs, kernel, strides, activation=True):
    x = layers.DepthwiseConv2D(kernel, strides=strides, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.PReLU(shared_axes=[1, 2])(x)
    return x

In [5]:
def bottleneck(inputs, out_channels, expansion, strides):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, in_channels * expansion, 1, 1)
    x = depthwise_conv_block(x, 3, strides)
    x = layers.Conv2D(out_channels, 1, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if strides == 1 and in_channels == out_channels:
        x = layers.Add()([inputs, x])
    return x

In [6]:
def MobileFaceNet(input_shape=(112, 112, 3), embedding_size=128, num_classes=None):
    inputs = keras.Input(shape=input_shape)
    x = conv_block(inputs, 64, 3, 2)
    x = depthwise_conv_block(x, 3, 1)
    x = bottleneck(x, 64, 2, 2)
    x = bottleneck(x, 64, 2, 1)
    x = bottleneck(x, 64, 2, 1)
    x = bottleneck(x, 64, 2, 1)
    x = bottleneck(x, 128, 4, 2)
    x = bottleneck(x, 128, 2, 1)
    x = bottleneck(x, 128, 2, 1)
    x = bottleneck(x, 128, 2, 1)
    x = bottleneck(x, 128, 2, 1)
    x = bottleneck(x, 128, 2, 1)
    x = bottleneck(x, 128, 2, 1)
    x = bottleneck(x, 128, 2, 1)
    x = conv_block(x, 512, 1, 1)
    x = layers.DepthwiseConv2D(7, 1, padding='valid', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(embedding_size)(x)
    x = layers.BatchNormalization()(x)
    if num_classes:
        x = layers.Dense(num_classes, activation='softmax')(x)
    model = keras.Model(inputs, x, name='MobileFaceNet')
    return model

In [7]:
# Data Preparation
data_dir = 'dataset'  # Path to your dataset folder
img_height, img_width = 112, 112
batch_size = 32
num_classes = len(next(os.walk(data_dir))[1])

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
)

In [8]:
train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 12724 images belonging to 30 classes.


In [14]:
# Save class_indices mapping
import json
with open('class_indices.json', 'w') as f:
    json.dump(train_gen.class_indices, f)

In [9]:
val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 3165 images belonging to 30 classes.


In [10]:
# Model Setup
model = MobileFaceNet(input_shape=(img_height, img_width, 3), embedding_size=128, num_classes=num_classes)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

2025-06-17 13:38:14.866297: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-06-17 13:38:14.866329: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-06-17 13:38:14.866335: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-06-17 13:38:14.866358: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-17 13:38:14.866371: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='accuracy',      # or 'accuracy' if you want to monitor training accuracy
    patience=5,                  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,   # Restore model weights from the epoch with the best value of the monitored quantity
    min_delta=0.001,             # Minimum change to qualify as an improvement
    verbose=1
)

In [12]:
# Training
model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen
)

/Users/nishantkumar/Desktop/Facial Recognition/facenet_env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


2025-06-17 13:38:21.565354: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.2598 - loss: 2.8701

/Users/nishantkumar/Desktop/Facial Recognition/facenet_env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


398/398 ━━━━━━━━━━━━━━━━━━━━ 235s 523ms/step - accuracy: 0.2600 - loss: 2.8688 - val_accuracy: 0.1422 - val_loss: 4.8353
Epoch 2/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 201s 505ms/step - accuracy: 0.4842 - loss: 1.8102 - val_accuracy: 0.4085 - val_loss: 2.4689
Epoch 3/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 200s 502ms/step - accuracy: 0.5902 - loss: 1.4308 - val_accuracy: 0.4900 - val_loss: 1.7864
Epoch 4/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 203s 510ms/step - accuracy: 0.6326 - loss: 1.2691 - val_accuracy: 0.4629 - val_loss: 2.2694
Epoch 5/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 203s 509ms/step - accuracy: 0.6912 - loss: 1.0828 - val_accuracy: 0.5602 - val_loss: 1.5272
Epoch 6/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 200s 503ms/step - accuracy: 0.7190 - loss: 0.9820 - val_accuracy: 0.5820 - val_loss: 1.4926
Epoch 7/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 202s 507ms/step - accuracy: 0.7338 - loss: 0.9154 - val_accuracy: 0.5953 - val_loss: 1.4072
Epoch 8/50
398/398 ━━━━━━━━━━━━━━━━━━━━ 201s 504ms/step - accuracy: 0.7476 - loss: 0.87

In [13]:
# Save the model
model.save('mobilefacenet_trained.h5')
print('Training complete. Model saved as mobilefacenet_trained.h5')

Training complete. Model saved as mobilefacenet_trained.h5


In [15]:
# Save class_indices mapping for use during inference
import json
with open('class_indices.json', 'w') as f:
    json.dump(train_gen.class_indices, f)
print('Saved class_indices mapping to class_indices.json')

Saved class_indices mapping to class_indices.json
